In [1]:
from sklearn.datasets import load_digits

In [2]:
import numpy as np
from sklearn.externals import joblib

In [3]:
from sklearn.datasets import fetch_olivetti_faces

In [4]:
faces = fetch_olivetti_faces()

In [5]:
faces.images.shape

(400, 64, 64)

In [6]:

#X, y = load_digits(n_class=1,return_X_y=True)
X,y = faces.images , faces.target

In [7]:
from PIL import Image

In [8]:
def showdigit(index,multiplier=70,size=200):
    tstimage1= Image.fromarray(X[index].reshape((8,8)) * multiplier)
    tstimage1 = tstimage1.convert('L')
    tstimage1 = tstimage1.resize((size,size))
    tstimage1.show()

In [9]:
def showpic(digit,multiplier=70,size=200):
    tstimage1= Image.fromarray(digit.reshape((64,64)) * multiplier)
    tstimage1 = tstimage1.convert('L')
    tstimage1 = tstimage1.resize((size,size))
    tstimage1.show()

In [20]:
from keras.models import Model, Sequential
from keras.layers import Flatten,Dropout, Dense, Input,merge
from keras.layers.convolutional import Convolution2D, Deconvolution2D, UpSampling2D, MaxPooling2D
from keras.layers.core import Reshape

In [21]:
#model=joblib.load("Sigmoid20epoch5gen.pkl")

In [22]:
from math import sqrt
PIC_SHAPE = 64*64

gen_input = Input(shape=(PIC_SHAPE,))
orig_input = Input(shape=(PIC_SHAPE,))

generator = Reshape((1, int(sqrt(PIC_SHAPE)), int(sqrt(PIC_SHAPE))),name = "generator11")(gen_input)
generator = Convolution2D(1, 12, 12, border_mode='same',name = "generator10")(generator)#Dense(64*64,activation = 'sigmoid',name = "encoder_1")
generator = Flatten(name = "generator2")(generator)
generator = Dense(64*64,activation = 'sigmoid',name = "encoder_1")(generator)
generator = Dense(64*32,activation = 'sigmoid',name = "encoder_2")(generator)
generator = Dense(64*40,activation = 'sigmoid',name = "encoder_3")(generator)
gen_out = Dense(PIC_SHAPE,activation = 'sigmoid',name = "generator1")(generator)
#generator4= Deconvolution2D(3, 3, 3, output_shape=(None, 3, 14, 14), border_mode='valid')(generator3)
#gen_out1 = generator(gen_input)
#gen_out2 = generator2(gen_out1)
#gen_out = generator3(gen_out2)

merged_vector = merge([orig_input,gen_out], mode='sum',output_shape=(PIC_SHAPE,))

predictor = Reshape((1, int(sqrt(PIC_SHAPE)), int(sqrt(PIC_SHAPE))),name = "predictor11")(merged_vector)
predictor = Convolution2D(1, 12, 12, border_mode='same',name = "predictor10")(predictor)
predictor = Flatten(name = "predictor2")(predictor)
predictor = Dense(64*32, activation='sigmoid',name="predictor_1")(predictor)
predictor = Dense(64, activation='sigmoid',name="predictor_2")(predictor)
predictions = Dense(1, activation='sigmoid',name="predictor_3")(predictor)

model = Model(input=[gen_input,orig_input], output=predictions)

In [23]:
startwith = joblib.load("modelFACES_LAST_epoch.pkl")
model.set_weights(startwith)
joblib.dump(startwith,"start")

Exception: Layer weight shape (4096, 4096) not compatible with provided weight shape (4096, 2048)

In [24]:
model.layers#[4].trainable

In [25]:
def generator_error(y_true,y_pred):
    #y_pred = np.array(y_pred)
    return np.log(np.ones(y_pred.shape)-y_pred).sum()

In [26]:
from theano.tensor import basic as tensor, subtensor
def generator_error_theano(y_true,y_pred):
    '''
    TODO write normalisation
    '''
    return -tensor.log(1.0 - y_pred)  #-(target * tensor.log(output) + (1.0 - target) *
    #return -tensor.log(1.0 + (y_pred-1.0))

In [27]:
def tester_error(y_true,y_pred):
    y_pred = np.array(y_pred)
    #log(pred) if true = 0
    #log(1-pred) if true=1
    return np.log(y_true - y_pred*((-1)**(y_true+1))).sum()

In [28]:
def tester_error_theano(target,output):
    '''
    y_true = 0 -> it was real
    '''
    
    return -((target) * tensor.log(output) + (1.0 - target) * tensor.log(1.0 - output))



In [29]:
def generate_input(length=64,bottom_border =0 ,top_border=1):
    '''
    generates list of length length with trash from custom distribution.
    '''
    return 0

In [30]:
npx = np.array(X)

In [31]:
showpic(X[70])


In [32]:
bottom = np.array(X).min()
top = np.array(X).max()

In [33]:
X = X/top

In [34]:
top

1.0

In [35]:
generator_layers = 3
tester_train_steps = 1
n_epoch = 100
xnum=0

In [36]:
from keras.optimizers import SGD

opt = "adam"#SGD(lr = 0.0001)

In [37]:
#showdigit(0,multiplier=1)

In [ ]:
import theano.sandbox.cuda
theano.sandbox.cuda.use("gpu")

from tqdm import tqdm
for epoch in range(0,n_epoch):
    
    print("Epoch ",epoch)#,"Iter ",xnum)
    joblib.dump(model.get_weights(),"modelFACES_LAST_epoch.pkl",compress=9)
    for xnum in tqdm(range(0,len(X))):
        

        #unfreezing tester
        #model.layers[-1].trainable = True
        for i in model.layers:
            if "predictor" in i.name:
                i.trainable = True
        #freezing generator
        for i in model.layers:
            if ("encoder" in i.name or "decoder" in i.name or "generator" in i.name):

                i.trainable = False
        model.compile(opt , loss = tester_error_theano)
        #doing tester_train_steps of tester training
        for k in range(0,tester_train_steps):
            #for xnum in range(0,len(X)):
            model.train_on_batch([np.zeros((PIC_SHAPE,)).reshape((-1,PIC_SHAPE)),np.array(X[xnum]).reshape((-1,PIC_SHAPE))],np.array([0.0])) #inputs original image
            model.train_on_batch([np.random.uniform(bottom,top,PIC_SHAPE).reshape((-1,PIC_SHAPE)),np.zeros((PIC_SHAPE,)).reshape((-1,PIC_SHAPE))],np.array([1.0])) #inputs noise to generator
            

        #Unfreezing generator        
        for i in model.layers:
            if ("encoder" in i.name or "decoder" in i.name or "generator" in i.name):
                i.trainable = True
        #freezing tester
        #model.layers[-1].trainable = False
        for i in model.layers:
            if "predictor" in i.name:
                i.trainable = False
                
        model.compile(opt, loss = generator_error_theano)
        #doing ? steps of generator training
        #print("Ill train discriminator")
        for nn in range(0,2):
            model.train_on_batch([np.random.uniform(bottom,top,PIC_SHAPE).reshape((-1,PIC_SHAPE)),np.zeros((PIC_SHAPE,)).reshape((-1,PIC_SHAPE))],np.array([1.0])) #inputs noise to generator

Epoch  0


100%|██████████| 400/400 [35:59<00:00,  5.24s/it]


Epoch  1


100%|██████████| 400/400 [36:43<00:00,  5.62s/it]


Epoch  2


...

100%|██████████| 400/400 [37:08<00:00,  5.38s/it]


Epoch  4


100%|██████████| 400/400 [37:13<00:00,  5.44s/it]


Epoch  5


 62%|██████▏   | 246/400 [22:06<13:35,  5.29s/it]

In [39]:
pred = model.predict([np.random.uniform(bottom,top,PIC_SHAPE).reshape((-1,PIC_SHAPE)),np.zeros((PIC_SHAPE,)).reshape((-1,PIC_SHAPE))])

In [40]:
#import subprocess subprocess.open (args 2 pipes -> -<)
pred2 = model.predict([np.zeros((PIC_SHAPE,)).reshape((-1,PIC_SHAPE)),np.array(X[44]).reshape((-1,PIC_SHAPE))])

In [41]:
pred2

array([[  5.15669262e-06]], dtype=float32)

In [42]:
for i in range(len(model.layers)):
            if ("encoder" in model.layers[i].name or "decoder" in model.layers[i].name or "generator" in model.layers[i].name):
                last_gen = i

In [43]:
last_gen

7

In [44]:
import theano
get_activations = theano.function([model.layers[0].input], model.layers[last_gen].output, allow_input_downcast=True)
#activations = get_activations(X_batch) # same result as above

In [45]:
tst = np.random.uniform(bottom,top,PIC_SHAPE).reshape((-1,PIC_SHAPE))
showpic(tst,size=500,multiplier = 100)
Generated = get_activations(tst)
showpic(Generated,size=500,multiplier = 100)
showpic(X[31],size=500,multiplier = 100)
#showdigit(np.random.randint(low=0,high=120),size=500,multiplier = 20)

In [ ]:
joblib.dump(model,"Sigmoid500epoch3gen.pkl",compress=9)

In [18]:
from theano import function, config, shared, sandbox
import theano.tensor as T
import numpy
import time

#import os
#os.environ["THEANO_FLAGS"] = "mode=FAST_RUN,device=gpu0,floatX=float64"

import theano.sandbox.cuda
theano.sandbox.cuda.use("gpu")

vlen = 10 * 30 * 768  # 10 x #cores x # threads per core
iters = 1000

rng = numpy.random.RandomState(22)
x = shared(numpy.asarray(rng.rand(vlen), config.floatX))
f = function([], T.exp(x))
print(f.maker.fgraph.toposort())
t0 = time.time()
for i in range(iters):
    r = f()
t1 = time.time()
print("Looping %d times took %f seconds" % (iters, t1 - t0))
print("Result is %s" % (r,))
if numpy.any([isinstance(x.op, T.Elemwise) for x in f.maker.fgraph.toposort()]):
    print('Used the cpu')
else:
    print('Used the gpu')

Using gpu device 0: GeForce GTX 860M (CNMeM is disabled, cuDNN 4007)


[GpuElemwise{exp,no_inplace}(<CudaNdarrayType(float32, vector)>), HostFromGpu(GpuElemwise{exp,no_inplace}.0)]
Looping 1000 times took 0.359734 seconds
Result is [ 1.23178029  1.61879349  1.52278066 ...,  2.20771813  2.29967761
  1.62323296]
Used the gpu


In [ ]:
from math import sqrt
PIC_SHAPE = 64*64

gen_input = Input(shape=(PIC_SHAPE,))
orig_input = Input(shape=(PIC_SHAPE,))

generator = Reshape((1, int(sqrt(PIC_SHAPE)), int(sqrt(PIC_SHAPE))),name = "generator11")(gen_input)
generator = Convolution2D(1, 12, 12, border_mode='same',name = "generator10")(generator)#Dense(64*64,activation = 'sigmoid',name = "encoder_1")
#generator = Convolution2D(128, 3, 3, border_mode='same',name = "generator9")(generator)
#generator = MaxPooling2D((2,2),name = "generator8")(generator)
#generator = Convolution2D(32, 3, 3,name = "generator7")(generator)
#generator = Deconvolution2D(1, 3, 3, output_shape=(None, 1, 14, 14),subsample=(2,2), border_mode='valid',name = "generator6")(generator)
generator = UpSampling2D(size=(2, 2), dim_ordering='default',name = "generator5")(generator)
#generator = Deconvolution2D(1, 3, 3, output_shape=(None, 1, 32, 32), subsample=(2,2), border_mode='valid',name = "generator4")(generator)
generator = Deconvolution2D(1, 12, 12, output_shape=(None, 1, 33, 33),subsample=(2,2), border_mode='same',name = "generator3")(generator)
generator = Flatten(name = "generator2")(generator)
#generator1 = Dense(64*64,activation = 'sigmoid',name = "encoder_1")(gen_input)
#generator2 = Dense(64*32,activation = 'sigmoid',name = "encoder_2")(generator1)
#generator3 = Dense(64*32,activation = 'sigmoid',name = "encoder_3")(generator2)
gen_out = Dense(PIC_SHAPE,activation = 'sigmoid',name = "generator1")(generator)
#generator4= Deconvolution2D(3, 3, 3, output_shape=(None, 3, 14, 14), border_mode='valid')(generator3)
#gen_out1 = generator(gen_input)
#gen_out2 = generator2(gen_out1)
#gen_out = generator3(gen_out2)

merged_vector = merge([orig_input,gen_out], mode='sum',output_shape=(PIC_SHAPE,))

predictor1 = Dense(64*32, activation='sigmoid',name="predictor_1")(merged_vector)
predictor2 = Dense(64, activation='sigmoid',name="predictor_2")(predictor1)
predictions = Dense(1, activation='sigmoid',name="predictor_3")(predictor2)

model = Model(input=[gen_input,orig_input], output=predictions)